In [1]:
import pandas as pd
from pyne.material import Material as m
from pyne import nucname
import numpy as np

/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
names = [
        'assembly_id',
        'reactor_id',
        'reactor_type',
        'initial_uranium_kg',
        'initial_enrichment',
        'discharge_burnup',
        'discharge_date',
        'discharge_time',
        'total_assembly_decay_heat_kw',
        'name',
        'evaluation_date',
        'total_mass_g',
        'total_radioactivity_curies']
df = pd.read_csv('SNF_nuclide_2020_07_01.dat',
                 sep='\s+',
                 index_col=False,
                 names=names)

In [3]:
keep = df[['assembly_id','reactor_id','reactor_type','discharge_date','total_assembly_decay_heat_kw','name','total_mass_g']]

In [4]:
#BWRs 
BWR = keep.loc[keep['reactor_type'] == 'BWR']
# keep first BWR 
BWR_1 = keep.loc[keep['reactor_id'] == 122]
# keep first assembly
BWR_2 = keep.loc[keep['assembly_id'] == 254897]
heat = keep.groupby((df["assembly_id"] != df["assembly_id"].shift()).cumsum().values).first()
heat = heat[['assembly_id','total_assembly_decay_heat_kw']]

In [5]:
def decay_assembly(assembly_id): 
    assem = keep.loc[keep['assembly_id'] == assembly_id]
    compdict = {}
    for index, row in assem.iterrows():
        if row['assembly_id'] == assembly_id: 
            nuc = row['name']
            mass = row['total_mass_g']
            if nuc == 'ag-108m' or nuc == 'am-242m' or nuc == 'ag-110m': # meta stable nucs dont work?
                compdict = compdict 
            else:
                compdict[nucname.id(nuc)] = mass
    new_m = m(compdict,mass = -1)
    decayheat = new_m.decay_heat()
    decayheat_total = 0
    for key,val in decayheat.items(): 
        decayheat_total += val*10**3
    
    return decayheat_total 

In [ ]:
n = 244887
newheat = heat.head(n)
calc_heat = np.zeros(n)
perc_diff = np.zeros(n)
i = 0
for index, row in newheat.iterrows():
    calc_heat[i] = decay_assembly(row['assembly_id'])
    perc_diff[i] = calc_heat[i]/row['total_assembly_decay_heat_kw']*100
    i +=1
newheat['calc_heat'] = calc_heat
newheat['perc_diff'] = perc_diff

In [ ]:
newheat

In [ ]:
# To store once created
newheat.to_pickle(decayheat)

In [ ]:
# To load picked files into this notebook
newheat = pd.read_pickle(decayheat)